In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input{font-family:Consolas; font-size:12pt;}
div.prompt {min width:70px;}
div#toc-wrapper {padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe {font-size:12px;}
</style>
"""))

- red wine 품질 등급 에측

```
1. 데이터셋 확보 & 전처리
    독립변수와 타겟변수 분리 -> 독립변수 스케일조정(StandardScaler)
    ->타겟변수 원핫인코딩(get_dummies)-> 훈련셋과 테스트셋분리(train_test_split)
2. 모델 구성 (입력 11, 출력 ?)
3. 모델학습과정 설정(다중분류로 설정)
4. 모델학습(callbacks 이용)
5. 모델평가 - 그래프, 평가(테스트셋), 교차표/혼동행렬
6. 모델 저장 /사용
```

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint
from tensorflow.keras import metrics

# 1. 데이터셋 확보 & 전처리
- 독립변수와 타겟변수 분리 -> 독립변수 스케일조정(StandardScaler) ->타겟변수 원핫인코딩(get_dummies)-> 훈련셋과 테스트셋분리(train_test_split)

In [5]:
# 데이터 읽어오기
# np.loadtxt('data/winequality-red.csv', delimiter=';', skiprows=1)
# np.genfromtxt('data/winequality-red.csv', delimiter=';', skiprows=1)
redwine = pd.read_csv('data/winequality-red.csv', sep=';')
redwine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


- 데이터 열 설명

```
fixed acidity (고정 산도): 주로 주석산 농도를 나타냅니다.
volatile acidity (휘발성 산도): 아세트산 농도를 나타내며, 높을수록 와인의 향에 부정적인 영향을 줄 수 있습니다.
citric acid (구연산): 와인에 신선함을 더하는 역할을 합니다.
residual sugar (잔류 당분): 발효 후 남은 설탕의 양으로, 와인의 단맛에 기여합니다.
chlorides (염화물): 와인의 짠맛과 관련이 있습니다.
free sulfur dioxide (유리 이산화황): 와인을 오래 보관하게 하는 방부제 역할을 합니다.
total sulfur dioxide (총 이산화황): 유리 이산화황을 포함한 전체 이산화황의 양입니다.
density (밀도): 알코올 도수 및 당도와 관련이 있습니다.
pH: 산도(acidity) 수준을 나타냅니다.
sulphates (황산염): 와인의 황 함량을 나타내며, 방부 및 항산화 작용을 합니다.
alcohol (알코올): 와인의 알코올 도수입니다.
quality (품질): 전문가가 평가한 최종 품질 점수 (0~10점). 
```

In [6]:
redwine.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [7]:
#타겟변수의 균형 (불균등)
redwine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [12]:
#독립변수, 타겟변수 분리
X_redwine = redwine.iloc[:, :-1].values # to_numpy와 유사, numpy 배열로
y_redwine = redwine.iloc[:, -1]
X_redwine.shape, y_redwine.shape

((1599, 11), (1599,))

In [15]:
# 독립변수 X의 스케일 조정
scaler = StandardScaler()
scaled_X_redwine = scaler.fit_transform(X_redwine)
# 다중분류를 위한 원핫인코딩 -> numpy 배열로 변환
Y_redwine = pd.get_dummies(y_redwine).to_numpy()


In [17]:
# 독립변수, 종속변수
scaled_X_redwine.shape, Y_redwine.shape

((1599, 11), (1599, 6))